Creating a Twitter bot that uses Twitter's streaming API.  This Twitter bot listens to Tweets sent by users you choose. When the selected user(s) send a Tweet the Twitter bot then sends an automated response Tweet.  This particular Bot uses the Python package Tweepy to send the responses.  I hosted this Bot on Amazon Web Services utilizing their free services for new customers.

In [1]:
import ConfigParser
import json
import random, time, os

from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy import API

Keep Twitter API credentials in a config file.  Use Python's ConfigParser package to retrieve the API and Token credentials for communicating with the Twitter API. Config file stored on AWS instance along with main script.

In [ ]:
config = ConfigParser.ConfigParser() 
config.read('YOUR CONFIG FILE') 

consumer_key = config.get('API', 'key')
consumer_secret = config.get('API', 'secret')
access_token = config.get('TOKEN', 'key')
access_token_secret = config.get('TOKEN', 'secret')

Keep two Python lists with reply messages and hashtags. The script will choose a message and hashtag randomly from these two lists to send in the Tweet.

In [ ]:
msg_list = ["Your Messages"]
hashtags = ["#YourHashtags"]

In [ ]:
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
twitterApi = API(auth)
users = ["YOUR USERS"]

The code below is the main code needed for retrieving Tweets and responding. The script sends an image along with the main Tweet text in 25% of situations based on a randomly generated number.

In [ ]:
class ReplyToTweet(StreamListener):

    def on_status(self, data):
        try:
            tweet = json.loads(json.dumps(data._json)) #status object to json     
            tweetId = tweet["id"]
            screenName = tweet["user"]["screen_name"]
            if screenName == "YOUR USER":
                msg = "YOUR USER" + " "+ random.choice(msg_list) + random.randint(1, 6) *"!" + " " + random.choice(hashtags)
                picture = random.random()                
                if picture < 0.25: #25% chance of sending a Picture with tweet
                    pic = random.choice(os.listdir("/home/ubuntu/FOLDER"))
                    pic_path = "/home/ubuntu/FOLDER/"  +  pic
                    twitterApi.update_with_media(status=msg, in_reply_to_status_id=tweetId, filename=pic_path)
                else:
                    twitterApi.update_status(status=msg, in_reply_to_status_id=tweetId)
        except:
            pass

    def on_error(self, status):
        time.sleep(5)
        return True
    
if __name__ == '__main__':
    streamListener = ReplyToTweet()
    twitterStream = Stream(auth, streamListener)
    twitterStream.userstream(_with=["YOUR USER"]) #stream following accounts